### House Price prediction With mlflow experiment tracking

• Run hyperparameter tuning while training a model

• Log every Hyperparameter and metrics in the Mlflow UI

• Compare the results of the various runs in the Mlflow UI

• Choose the best run and register it as a model

In [2]:
import pandas as pd
import mlflow as mfl
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor        # determines it's a regression problem (dealing with numerical data/ predict continuous values)
from sklearn.model_selection import train_test_split, GridSearchCV
from mlflow.models import infer_signature
from mlflow.models.signature import ModelSignature

In [3]:
housing = fetch_california_housing()
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [4]:
df = pd.DataFrame(housing.data, columns = housing.feature_names)      # input columns
df['Price'] = housing.target                                         # output column

df.head(10)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
5,4.0368,52.0,4.761658,1.103627,413.0,2.139896,37.85,-122.25,2.697
6,3.6591,52.0,4.931907,0.951362,1094.0,2.128405,37.84,-122.25,2.992
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25,2.414
8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26,2.267
9,3.6912,52.0,4.970588,0.990196,1551.0,2.172269,37.84,-122.25,2.611


> #### Define the **Train & Test** splits

In [5]:
X = df.drop(columns = ['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

> #### GridSearchCV
> #### Define the **hyperparameter grid**

In [24]:
# 1> Number of trees in random forest
n_estimators: list[int] = [20, 60, 100, 120]

# 2> Number of features to consider at every split
max_features: list[float] = [0.2, 0.6, 1.0]

# 3> Maximum number of levels in tree
max_depth = [2, 8, None]

# 4> Number of samples
max_samples: list[float] = [0.5, 0.75, 1.0]

# 5> Minimum number of samples required to split a node
min_samples_split = [2, 5]

# 6> Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]

hyper_params_grid: dict[str, object] = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'max_samples': max_samples,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf
}

print(hyper_params_grid)

{'n_estimators': [20, 60, 100, 120], 'max_features': [0.2, 0.6, 1.0], 'max_depth': [2, 8, None], 'max_samples': [0.5, 0.75, 1.0], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2]}


In [26]:
# create a function to perform hyper parameter tuning
def hyperparameter_tuning(X_train, y_train, params_grid) -> GridSearchCV[RandomForestRegressor]:
  rfreg_model = RandomForestRegressor()
  grid_search = GridSearchCV(estimator = rfreg_model,
                             param_grid = params_grid,
                             scoring = ['neg_mean_squared_error'],
                             cv = 10,
                             refit = 'neg_mean_squared_error',
                             verbose = 2,
                             n_jobs = -1)

  grid_search.fit(X_train, y_train)
  return grid_search

In [27]:
try:
	mfl.set_experiment('RandomForest Regressor - Housing data')

except mfl.exceptions.MlflowException as exception:
	print(f'Failed to set experiment: {exception}')

In [18]:
mfl.get_tracking_uri()          # If suppose "tracking_uri" wasn't set, "tracking_url_type_store" next always returns a 'file'

'file:///content/mlruns'

In [19]:
from urllib.parse import urlparse
tracking_url_type_store = urlparse(mfl.get_tracking_uri()).scheme         # prints 'file'

In [20]:
signature: ModelSignature = infer_signature(X_train, y_train)

In [ ]:
try:
    with mfl.start_run() as rf_run:
      grid_search = hyperparameter_tuning(X_train, y_train, hyper_params_grid)

      # get the best model after hyper-parameter tuning
      best_model: RandomForestRegressor = grid_search.best_estimator_

      # perform predictions on the test set with the best model
      y_pred = best_model.predict(X_test)

      # Calculate mse metric & log it
      mse = mean_squared_error(y_test, y_pred)
      mfl.log_metric('mean squared error is:', mse)

      # log the best-hyperparams
      best_params = grid_search.best_params_
      mfl.log_params(best_params)

      # log the best model
      if tracking_url_type_store != 'file':
        mfl.sklearn.log_model(best_model,
                              artifact_path = 'final_model',
                              registered_model_name = 'RandomForest_Regressor_housing_price_prediction_model')
      # when "tracking_uri" is not set
      else:
        mfl.sklearn.log_model(best_model,
                              artifact_path = 'final_model',
                              signature = signature)

except ValueError as error:
        print('Unexpected error: %s', error)
        raise

except mfl.exceptions.MlflowException as exception:
    print(f'Failed to start run or log metrics: {exception}')

Fitting 10 folds for each of 432 candidates, totalling 4320 fits


> ### mlflow start server with port -command line

In [23]:
!mlflow server --port 5000

[2025-03-05 14:06:11 +0000] [5237] [INFO] Starting gunicorn 23.0.0
[2025-03-05 14:06:11 +0000] [5237] [INFO] Listening at: http://127.0.0.1:5000 (5237)
[2025-03-05 14:06:11 +0000] [5237] [INFO] Using worker: sync
[2025-03-05 14:06:11 +0000] [5238] [INFO] Booting worker with pid: 5238
[2025-03-05 14:06:11 +0000] [5243] [INFO] Booting worker with pid: 5243
[2025-03-05 14:06:11 +0000] [5244] [INFO] Booting worker with pid: 5244
[2025-03-05 14:06:11 +0000] [5245] [INFO] Booting worker with pid: 5245
[2025-03-05 14:06:16 +0000] [5237] [INFO] Handling signal: int

Aborted!
[2025-03-05 14:06:16 +0000] [5244] [INFO] Worker exiting (pid: 5244)
[2025-03-05 14:06:16 +0000] [5238] [INFO] Worker exiting (pid: 5238)
[2025-03-05 14:06:16 +0000] [5245] [INFO] Worker exiting (pid: 5245)
[2025-03-05 14:06:16 +0000] [5243] [INFO] Worker exiting (pid: 5243)
[2025-03-05 14:06:17 +0000] [5237] [INFO] Shutting down: Master
